<a href="https://colab.research.google.com/github/ChabiMax/author_network/blob/main/Code/Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center><strong><font color="chillipepper">Réseau Wikipédia des auteurs francophones</font></strong></center></h1>
<h5><center><font color="green">Science des réseaux sociaux et économiques <h3><center>


<center>
Maxime Chabriel, Yasmine Houri, Mathis Sansu  

ENSAE 2022
</center>

<h2><center><strong><font color="chillipepper">Scraping des données</font></strong></center></h2>

# Packages

In [ ]:
from tqdm import *
import time
import requests
import bs4
from collections import Counter
import json
import os
from google.colab import files
import pickle as pickle
import pandas as pd
import numpy as np

# Scraping

In [ ]:
raw_data = {} # Dictionnaire pour stocker les données

In [ ]:
# URL de la page initiale de notre liste d'auteurs
URL = 'https://fr.wikipedia.org/wiki/Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise_par_ordre_chronologique'
response = requests.get(URL) # Sélectionne la page
soup = bs4.BeautifulSoup(response.text, 'html.parser') # Récupération du code html

In [ ]:
dictionary = {} # Déclaration d'un dictionnaire vide
# On initialise des attributs
date_naissance = None 
date_mort = None
name = 'init'
author_link = 'init'
id = 0
serie = 0
links = soup.find(id="bodyContent").select('a') # Récupération des hyperliens

In [ ]:
# Extraction de la liste d'auteurs et de leurs attributs

for link in links: # Boucle sur les hyperliens récupérés
  string = link.get_text()
  if link.get('href') != None:
    if not any(char.isdigit() for char in string) : 
      if serie == 2 :
        serie = 0
        date_mort = None
        dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}
      serie = serie+1
      id = id+1
      name = string
      date_naissance = None
      date_mort = None
      if 'https://' in link.get('href'):
        author_link = link.get('href')
      else:
        author_link = 'https://fr.wikipedia.org' + link.get('href')
    else :
      if serie == 1 :
        serie = serie+1
        date_naissance = string
      else :
        if serie == 2 :
          serie = 0
          date_mort = string
          dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}

In [ ]:
# Nettoyage manuel : certains lien récupérés ne correpondent pas à des auteurs

unwanted_list = [key for key in dictionary.keys() if 'modifier' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'siècle' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'ms.' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'Portail' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "Liste d'écrivains de langue française" in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "↑" in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "" == dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "Langue française et francophonie" in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "Chronologie de la littérature" in dictionary[key]['name']]

# Les commandes au-dessus permettent de récupérer les clés problématiques dans le dictionnaire

# On enlève donc les éléments associés à ces clés dans le dictionnaire
dictionary = dict((key, dictionary[key]) for key in dictionary.keys() if key not in unwanted_list)

In [ ]:
 # Extraction de la liste des genres littéraires (mêmes procédures)

URL = 'https://fr.wikipedia.org/wiki/Genre_litt%C3%A9raire'
response = requests.get(URL)
soup = bs4.BeautifulSoup(response.text, 'html.parser')

dictionary_genres = {}
name = 'init'
genre_link = 'init'
id = 0
links = soup.find(id="bodyContent").select('a')
serie = 0
for link in links:
  string = link.get_text()
  if link.get('href') != None:
    id = id+1
    name = string
    if 'https://' in link.get('href'):
      genre_link = link.get('href')
    else:
      genre_link = 'https://fr.wikipedia.org' + link.get('href')
    dictionary_genres[id] = {'name' : name, 'link' : genre_link}


In [ ]:
# Nettoyage manuel

unwanted_list = list(range(0,49))
unwanted_list += list(range(278,344))
unwanted_list += [key for key in dictionary_genres.keys() if 'modifier' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Portail' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Li Shangyin' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Sei Shōnagon' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Yoshida Kenkō' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Pascal Quignard' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Sévigné' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Caractères' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Bruyère' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Montandon' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if dictionary_genres[key]['link'] in [dictionary[key_bis]['link'] for key_bis in dictionary.keys()]]
unwanted_list += [key for key in dictionary_genres.keys() if any(char.isdigit() for char in dictionary_genres[key]['name'])]
unwanted_list += list(range(285,len(dictionary_genres.keys())))
dictionary_genres = dict((key, dictionary_genres[key]) for key in dictionary_genres.keys() if key not in unwanted_list)

In [ ]:
# Remise à zéro des clés pour bien indexer le dictionnaire

old_dictionary = dictionary.copy()
dictionary = {}
id = 0
for key in old_dictionary :
  dictionary[id] = old_dictionary[key]
  id += 1

old_dictionary_genres = dictionary_genres.copy()
dictionary_genres = {}
for key in old_dictionary_genres :
  dictionary_genres[id] = old_dictionary_genres[key]
  id += 1

In [ ]:
# Fonction réduisant une balise html de lien sous forme d'un lien wikipédia classique

def html_to_link(html_link) :
  if html_link.get('href') != None :
    if 'https://' in html_link.get('href'):
      html_link = html_link.get('href')
    else:
      html_link = 'https://fr.wikipedia.org' + html_link.get('href')
  else :
    html_link = ''
  return(html_link)

In [ ]:
# Création des listes des liens associés à nos pages Wikipédia (auteurs et genres)

author_links = [dictionary[key]['link'] for key in dictionary.keys()]
genre_links = [dictionary_genres[key]['link'] for key in dictionary_genres.keys()]

In [ ]:
# Parcours des pages et création du réseau d'auteurs

for id in tqdm_notebook(dictionary.keys()) :
  authtor_link = dictionary[id]['link']
  response = requests.get(authtor_link)
  soup = bs4.BeautifulSoup(response.text, 'html.parser')
  links = soup.find(id="bodyContent").select('a')
  network_aslink = []
  net_genre_aslink = []
  for link in links:
    link = html_to_link(link)
    if link in author_links :
      network_aslink.append(link)
    if link in genre_links :
      net_genre_aslink.append(link)
  network = []
  net_genre = []
  for link in network_aslink :
    for id_bis in dictionary.keys() :
      if link == dictionary[id_bis]['link'] :
        network.append(id_bis)
  for link in net_genre_aslink :
    for id_bis in dictionary_genres.keys() :
      if link == dictionary_genres[id_bis]['link'] :
        net_genre.append(id_bis)
  counted_network = Counter(network)
  counted_net_genre = Counter(net_genre)
  dictionary[id]['taille_html_article'] = len(str(soup))
  dictionary[id]['network_interauteur'] = list(counted_network.keys())
  dictionary[id]['poids_liens_interauteur'] = list(counted_network.values())
  dictionary[id]['genres_mentionnes'] = list(counted_net_genre.keys())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1584 [00:00<?, ?it/s]

# Formatage données

In [ ]:
raw_data1 = dictionary.copy()
for key in raw_data1.keys() :
  raw_data1[key]['type'] = 'auteur'

for key in dictionary_genres.keys() :
  raw_data1[key] = dictionary_genres[key].copy()
  raw_data1[key]['type'] = 'genre'

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'genre' :
    raw_data1[key_genre]['auteurs_du_genre'] = []
for key_auteur in raw_data1.keys() :
  if raw_data1[key_auteur]['type'] == 'auteur' :
    for genre in raw_data1[key_auteur]['genres_mentionnes'] :
      raw_data1[genre]['auteurs_du_genre'] = raw_data1[genre]['auteurs_du_genre'] + [key_auteur]

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'auteur' :
    try :
      raw_data1[key_genre]['date_naissance'] = int(raw_data1[key_genre]['date_naissance'])
    except :
      raw_data1[key_genre]['date_naissance'] = np.nan
    try :
      raw_data1[key_genre]['date_mort'] = int(raw_data1[key_genre]['date_mort'])
    except :
      raw_data1[key_genre]['date_mort'] = np.nan

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'genre' :
    raw_data1[key_genre]['debut_genre'] = min([raw_data1[auteur]['date_naissance'] for auteur in raw_data1 if raw_data1[auteur]['type'] == 'auteur' and key_genre in raw_data1[auteur]['genres_mentionnes']] + [9999])
    raw_data1[key_genre]['fin_genre'] = max([raw_data1[auteur]['date_mort'] for auteur in raw_data1 if raw_data1[auteur]['type'] == 'auteur' and key_genre in raw_data1[auteur]['genres_mentionnes']] + [0])

for key_auteur1 in tqdm_notebook(raw_data1.keys()) :
  if raw_data1[key_auteur1]['type'] == 'auteur' :
    raw_data1[key_auteur1]['contemporain'] = []
    raw_data1[key_auteur1]['poids_contempo_diff_naissance'] = []
    raw_data1[key_auteur1]['poids_contempo_annees_communes'] = []
    for key_auteur2 in raw_data1.keys() :
      if (raw_data1[key_auteur2]['type'] == 'auteur') & (key_auteur2 != key_auteur1) :
        mort_1 = [raw_data1[key_auteur1]['date_mort'] if raw_data1[key_auteur1]['date_mort'] != None else 2022 if raw_data1[key_auteur1]['date_mort'] == None and raw_data1[key_auteur1]['date_naissance'] > 1900 else None]
        mort_2 = [raw_data1[key_auteur2]['date_mort'] if raw_data1[key_auteur2]['date_mort'] != None else 2022 if raw_data1[key_auteur1]['date_mort'] == None and raw_data1[key_auteur2]['date_naissance'] > 1900 else None]
        naissance_1 = [raw_data1[key_auteur1]['date_naissance'] if raw_data1[id]['date_mort'] != None else None]
        naissance_2 = [raw_data1[key_auteur2]['date_naissance'] if raw_data1[key_auteur2]['date_mort'] != None else None]
        try:
          annees_1 = set(range(naissance_1[0],mort_1[0]))
          annees_2 = range(naissance_2[0],mort_2[0])
          annees_communes = list(annees_1.intersection(annees_2))
          if (mort_1[0] > naissance_2[0] and naissance_1[0] < mort_2[0]) or (mort_2[0] > naissance_1[0] and naissance_2[0] < mort_1[0]):
            raw_data1[key_auteur1]['contemporain'].append(key_auteur2)
            raw_data1[key_auteur1]['poids_contempo_diff_naissance'].append(abs(naissance_1[0] - naissance_2[0]))
            raw_data1[key_auteur1]['poids_contempo_annees_communes'].append(len(annees_communes))

        except :
          pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1786 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(raw_data1).T
df.head(50)

,name,link,date_mort,date_naissance,taille_html_article,network_interauteur,poids_liens_interauteur,genres_mentionnes,type,contemporain,poids_contempo_diff_naissance,poids_contempo_annees_communes,auteurs_du_genre,debut_genre,fin_genre
0,Turold,https://fr.wikipedia.org/wiki/Turold,NaN,NaN,73673,[],[],[],auteur,[],[],[],NaN,NaN,NaN
1,Philippe de Thaon,https://fr.wikipedia.org/wiki/Philippe_de_Thaon,NaN,NaN,68910,[],[],[],auteur,[],[],[],NaN,NaN,NaN
2,Benoît de Sainte-Maure,https://fr.wikipedia.org/wiki/Beno%C3%AEt_de_S...,NaN,NaN,83269,[4],[1],[1627],auteur,[],[],[],NaN,NaN,NaN
3,Jean Bodel,https://fr.wikipedia.org/wiki/Jean_Bodel,1165,NaN,73274,"[10, 22]","[1, 1]","[1663, 1691, 1604]",auteur,[],[],[],NaN,NaN,NaN
4,Wace,https://fr.wikipedia.org/wiki/Wace,1174,1100,99714,[6],[2],"[1591, 1662]",auteur,"[5, 6, 7, 9, 10, 11]","[10, 30, 50, 60, 60, 60]","[52, 44, 24, 14, 14, 14]",NaN,NaN,NaN
5,Eudes de Deuil,https://fr.wikipedia.org/wiki/Eudes_de_Deuil,1162,1110,68248,[],[],[1712],auteur,"[4, 6, 7, 9, 10, 11]","[10, 20, 40, 50, 50, 50]","[52, 32, 12, 2, 2, 2]",NaN,NaN,NaN
6,Chrétien de Troyes,https://fr.wikipedia.org/wiki/Chr%C3%A9tien_de...,1180,1130,207040,"[4, 732]","[1, 1]","[1628, 1712, 1663]",auteur,"[4, 5, 7, 9, 10, 11, 15]","[30, 20, 20, 30, 30, 30, 48]","[44, 32, 30, 20, 20, 20, 2]",NaN,NaN,NaN
7,Conon de Béthune,https://fr.wikipedia.org/wiki/Conon_de_B%C3%A9...,1219,1150,82584,[11],[2],[1712],auteur,"[4, 5, 6, 9, 10, 11, 15, 16, 17, 18, 19]","[50, 40, 20, 10, 10, 10, 28, 50, 50, 51, 60]","[24, 12, 30, 53, 59, 53, 41, 19, 19, 18, 9]",NaN,NaN,NaN
8,Alexandre de Bernay,https://fr.wikipedia.org/wiki/Alexandre_de_Bernay,NaN,1158,67230,[],[],[],auteur,[],[],[],NaN,NaN,NaN
9,Gace Brulé,https://fr.wikipedia.org/wiki/Gace_Brul%C3%A9,1213,1160,68980,[],[],[],auteur,"[4, 5, 6, 7, 10, 11, 15, 16, 17, 18, 19]","[60, 50, 30, 10, 0, 0, 18, 40, 40, 41, 50]","[14, 2, 20, 53, 53, 53, 35, 13, 13, 12, 3]",NaN,NaN,NaN


# Import des données

In [ ]:
# Import sous forme pickle (objet python assez léger)

import pickle
with open('raw_data.pkl', 'wb') as f:
    pickle.dump(raw_data1, f)
files.download('raw_data.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>